# 2023 Research Template

In [1]:
#import tools
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
import matplotlib.pyplot as plt
from array import *
import random

2023-08-10 17:57:31.420510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#import dataset & labels
x_data = np.loadtxt("/home/jovyan/SummerResearch/Data/CC400_X_corr.csv", delimiter = ',')
y_data = np.loadtxt("/home/jovyan/SummerResearch/Data/Y.csv", delimiter = ',')
x_data_sampled_train = np.empty((696,x_data.shape[1]))
x_data_sampled_val = np.empty((175,x_data.shape[1]))
y_data_sampled_train = np.empty(696)
y_data_sampled_val = np.empty(175)
type(x_data)
y_data.shape

(871,)

In [3]:
"""
#check for data normalization dynamic to the dimensions of the set
max = 0.0
min = 0.0
count = 0
temp = 0.0

for i in range(x_data.shape[0]):
    for j in range(x_data.shape[1]):
        temp = x_data[i][j]
        count += 1
        if temp > max:
            max = temp
        elif temp < min:
            min = temp

print("Checking for normalization of the data set...")
print("Max Value: {a:1.5f}".format(a = max))
print("Min Value: {a:1.5f}".format(a = min))
print("Elements Checked:", count)
"""

'\n#check for data normalization dynamic to the dimensions of the set\nmax = 0.0\nmin = 0.0\ncount = 0\ntemp = 0.0\n\nfor i in range(x_data.shape[0]):\n    for j in range(x_data.shape[1]):\n        temp = x_data[i][j]\n        count += 1\n        if temp > max:\n            max = temp\n        elif temp < min:\n            min = temp\n\nprint("Checking for normalization of the data set...")\nprint("Max Value: {a:1.5f}".format(a = max))\nprint("Min Value: {a:1.5f}".format(a = min))\nprint("Elements Checked:", count)\n'

In [4]:
#measure percent of positive vs negative samples in our set
true = 0
false = 0
true_rate = 0.0
false_rate = 0.0
#pos samples 403
#neg samples 468

for x in range(x_data.shape[0]):
    if y_data[x] == 1: 
        true += 1
    else:
        false += 1
    
true_rate = true/(true+false) * 100
false_rate = false/(true+false) * 100

print("Distribution of Subgroups in Population...")
print("ASD Positive: {a:2.2f}%".format(a = true_rate), true)
print("ASD Negative: {a:2.2f}%".format(a = false_rate), false)

Distribution of Subgroups in Population...
ASD Positive: 46.27% 403
ASD Negative: 53.73% 468


In [5]:
def stratifiedRandomSampling():
    #split into categories
    percentage = 0.0
    pos_count = 0
    total_count = 0
    pos_range = 403
    neg_range = 468
    y = 0
    z = 0
    random_number = 0
    
    pos_strata = np.empty((403,x_data.shape[1]), dtype=float)
    neg_strata = np.empty((468,x_data.shape[1]), dtype=float)
    
    for x in range(x_data.shape[0]):
        if y_data[x] == 1:
            pos_strata[y] = x_data[x]
            y += 1
        else:
            neg_strata[z] = x_data[x]
            z += 1
            
    #have selector so that starts with negative sample?
    #right now is guaranteed to start with positive sample
    #marker = random.randrange(start=0,stop=2)
          
    for x in range(696):
        if percentage <= 0.46:
            #add pos sample
            #generate random number 0-402
            rand_number = random.randrange(start=0,stop=pos_range)
            #add to test_data -> guaranteed label 1
            x_data_sampled_train[x] = pos_strata[random_number]
            y_data_sampled_train[x] = 1
            #remove record
            np.delete(pos_strata, random_number, axis=1)
            #calculate percentage
            pos_range -= 1
            pos_count += 1
            total_count += 1
            percentage = pos_count / total_count
        else:
            #add neg sample
            #generate random number 0-467
            rand_number = random.randrange(start=0,stop=neg_range)
            #add to test_data -> guaranteed label 0
            x_data_sampled_train[x] = neg_strata[random_number]
            y_data_sampled_train[x] = 0
            #remove record
            np.delete(neg_strata, random_number, axis=1)
            #calculate percentage
            neg_range -= 1
            total_count += 1
            percentage = pos_count / total_count
            
    percentage = 0.0
    pos_count = 0
    total_count = 0
            
    for x in range(175):
        if percentage <= 0.46:
            #add pos sample
            if pos_range == 0:
                x_data_sampled_val[x] = pos_strata[0]
                continue
            #generate random number 0-402
            rand_number = random.randrange(start=0,stop=pos_range)
            #add to test_data -> guaranteed label 1
            x_data_sampled_val[x] = pos_strata[random_number]
            y_data_sampled_val[x] = 1
            #remove record
            np.delete(pos_strata, random_number, axis=1)
            #calculate percentage
            pos_range -= 1
            pos_count += 1
            total_count += 1
            percentage = pos_count / total_count
        else:
            #add neg sample
            if neg_range == 0:
                x_data_sampled_val[x] = neg_strata[0]
                continue
            #generate random number 0-467
            rand_number = random.randrange(start=0,stop=neg_range)
            #add to test_data -> guaranteed label 0
            x_data_sampled_val[x] = neg_strata[random_number]
            y_data_sampled_val[x] = 0
            #remove record
            np.delete(neg_strata, random_number, axis=1)
            #calculate percentage
            neg_range -= 1
            total_count += 1
            percentage = pos_count / total_count
        
    print(x_data_sampled_train.shape,x_data_sampled_val.shape,y_data_sampled_train.shape,y_data_sampled_val.shape)
    # when loop is done, remaining samples are copied to validation data
    return
    

In [6]:
def modelConstructor(n_hidden_units, n_hidden_layers):
    y = x = keras.layers.Input(shape=x_data_sampled_train.shape[1:])
    y = keras.layers.Flatten()(y)
    y = keras.layers.Dense(n_hidden_units)(y)
    for _ in range(n_hidden_layers):
        y_resid = y
        y = keras.layers.LayerNormalization()(y)
        y = keras.layers.Dense(n_hidden_units,
                               activation=keras.activations.relu)(y)
        y = keras.layers.Add()([y,y_resid])
    y = keras.layers.Dense(len(np.unique(y_data_sampled_train)),
                               activation=keras.activations.softmax)(y)
    model = keras.Model(x,y)
    model.summary()
    keras.utils.plot_model(model,show_shapes=True,expand_nested=True)
    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    optimizer=keras.optimizers.SGD(learning_rate=0.001))
    return model

In [7]:
#define data structure to hold 10x results

In [8]:
stratifiedRandomSampling()

(696, 77028) (175, 77028) (696,) (175,)


In [9]:
#y_data_sampled_train

In [10]:
counter_train = 0
counter_val = 0

for x in range(y_data_sampled_train.shape[0]):
    if y_data_sampled_train[x] == 1:
        counter_train += 1

for x in range(y_data_sampled_val.shape[0]):
    if y_data_sampled_val[x] == 1:
        counter_val += 1

print("Positive Training Count: ", counter_train)
print("Negative Training Count: ", 696-counter_train)
print("Positive Validation Count: ", counter_val)
print("Negative Validation Count: ", 175-counter_val)

Positive Training Count:  320
Negative Training Count:  376
Positive Validation Count:  79
Negative Validation Count:  96


In [ ]:
correct = 0
total = 175
labels_accuracy = 0.0
labels_accuracy_storage = np.empty(10,dtype=float)
results = np.empty((10,4), dtype=float)

#10 times cross validation
for x in range(10):
    print("Iteration:", x)
    stratifiedRandomSampling()
    model = modelConstructor(4,4)
    history = model.fit(x_data_sampled_train, y_data_sampled_train,
                epochs=3,
                verbose=2,
                batch_size=64,
                validation_data=(x_data_sampled_val,y_data_sampled_val))
    
    loss = history.history['loss']
    accuracy = history.history['sparse_categorical_accuracy']
    val_loss = history.history['val_loss']
    val_accuracy = history.history['val_sparse_categorical_accuracy']
    
    #get final results using last indexes
    results[x][0] = loss[-1]
    results[x][1] = accuracy[-1]
    results[x][2] = val_loss[-1]
    results[x][3] = val_accuracy[-1]
    
    output = model.predict(x_data_sampled_val[0:])
    np.argmax(output,axis=1)
    
    correct = 0
    
    for b in range(175):
        temp1 = output[b][0]
        temp2 = y_data_sampled_val[b]
        if temp1 == temp2:
            correct += 1
    
    labels_accuracy = correct/total
    labels_accuracy_storage[x] = labels_accuracy
    
loss_avg = 0.0
accuracy_avg = 0.0
val_loss_avg = 0.0
val_accuracy_avg = 0.0
labels_accuracy_avg = 0.0

for a in range(10):
    loss_avg += results[x][0]
for a in range(10):
    accuracy_avg += results[x][1]
for a in range(10):
    val_loss_avg += results[x][2]
for a in range(10):
    val_accuracy_avg += results[x][3]
    
loss_avg = loss_avg / 10
accuracy_avg = accuracy_avg / 10 * 100
val_loss_avg = val_loss_avg / 10
val_accuracy_avg = val_accuracy_avg / 10 * 100

for a in range(10):
    labels_accuracy_avg += labels_accuracy_storage[x]
    
labels_accuracy_avg = labels_accuracy_avg / 10 * 100

print("Training Loss Average: {a:1.5f}".format(a = loss_avg))
print("Training Accuracy Average: {a:1.2f}%".format(a = accuracy_avg))
print("Validation Loss Average: {a:1.5f}".format(a = val_loss_avg))
print("Validation Accuracy Average: {a:1.5f}%".format(a = val_accuracy_avg))
print("Label Accuracy Average: {a:1.2f}%".format(a = labels_accuracy_avg))

Iteration: 0
(696, 77028) (175, 77028) (696,) (175,)


2023-08-10 18:02:25.661003: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-08-10 18:02:25.661059: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-10 18:02:25.661130: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-hwb2i): /proc/driver/nvidia/version does not exist
2023-08-10 18:02:25.661488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 77028)]      0           []                               
                                                                                                  
 flatten (Flatten)              (None, 77028)        0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 4)            308116      ['flatten[0][0]']                
                                                                                                  
 layer_normalization (LayerNorm  (None, 4)           8           ['dense[0][0]']                  
 alization)                                                                                   

In [ ]:
output